In [12]:
%run data.ipynb

In [13]:
import pandas as pd
import glob

df_wise_reconciliation = df_wise.copy()
df_wise_reconciliation = df_wise_reconciliation[df_wise_reconciliation['wise.status'] != 'funds_refunded']

print(df_wise_reconciliation.info())
df_wise_reconciliation.head()

<class 'pandas.core.frame.DataFrame'>
Index: 3410 entries, 575093811 to 914901258
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   wise.id                     3410 non-null   int64              
 1   wise.user                   3410 non-null   int64              
 2   wise.targetAccount          3410 non-null   int64              
 3   wise.sourceAccount          3374 non-null   object             
 4   wise.quote                  0 non-null      object             
 5   wise.quoteUuid              3410 non-null   object             
 6   wise.status                 3410 non-null   object             
 7   wise.reference              3410 non-null   object             
 8   wise.rate                   3410 non-null   float64            
 9   wise.created                3410 non-null   datetime64[ns, UTC]
 10  wise.business               3410 non-null   int64   

,wise.id,wise.user,wise.targetAccount,wise.sourceAccount,wise.quote,wise.quoteUuid,wise.status,wise.reference,wise.rate,wise.created,wise.business,wise.transferRequest,wise.details.reference,wise.hasActiveIssues,wise.sourceCurrency,wise.sourceValue,wise.targetCurrency,wise.targetValue,wise.customerTransactionId
575093811,575093811,6421549,298725929,59807023,None,be48f661-368c-430b-82f9-94a101390e59,outgoing_payment_sent,,1.00000,2023-01-02 00:07:08+00:00,4556280,None,,False,GBP,46.86,GBP,46.86,72d112b5-775a-416e-a0d0-df6219c9d708
575919595,575919595,6421549,219325936,59807023,None,21416ea6-87fd-4bbd-8a64-2082c6894abc,outgoing_payment_sent,,1.00000,2023-01-02 23:47:59+00:00,4556280,None,,False,USD,1550.00,USD,1550.00,9e7ca373-3eb6-45a2-9403-9e9d9f786410
575919769,575919769,6421549,333432475,59807023,None,5f8c9bc9-0e3e-4748-9ef1-945507d73bb9,outgoing_payment_sent,,0.93705,2023-01-02 23:48:25+00:00,4556280,None,,False,USD,71.79,EUR,67.27,f4d246b7-5c51-4430-9698-54e8750fd3c9
575919989,575919989,6421549,333432571,59807023,None,778efd01-6d36-4b01-994a-253d2c064655,outgoing_payment_sent,,0.93720,2023-01-02 23:49:02+00:00,4556280,None,,False,USD,500.00,EUR,468.60,b2abdbb0-2f61-4dc4-bad5-bd04cd0cdaa6
575920084,575920084,6421549,266354361,59807023,None,59ff5aba-aa30-4fed-ad78-4160aca5dc60,outgoing_payment_sent,,1.00000,2023-01-02 23:49:16+00:00,4556280,None,,False,USD,265.00,USD,265.00,1841e526-f886-42d4-9cd4-c1afc5d017e9


In [14]:
df_platform_wise_related = df_platform[(df_platform['kind'] == 'EXPENSE') & (df_platform['payoutMethod.type'] == 'BANK_ACCOUNT') & (df_platform['type'] == 'DEBIT') & (df_platform['isRefunded'] == False)]
print(df_platform_wise_related.info())
df_platform_wise_related.head()

<class 'pandas.core.frame.DataFrame'>
Index: 3377 entries, 3986 to 819757
Data columns (total 74 columns):
 #   Column                                Non-Null Count  Dtype              
---  ------                                --------------  -----              
 0   id                                    3377 non-null   object             
 1   uuid                                  3377 non-null   object             
 2   kind                                  3377 non-null   object             
 3   payoutMethod                          0 non-null      float64            
 4   giftCardEmitterAccount                0 non-null      float64            
 5   group                                 3377 non-null   object             
 6   type                                  3377 non-null   object             
 7   description                           3377 non-null   object             
 8   createdAt                             3377 non-null   datetime64[ns, UTC]
 9   merchantId         

,id,uuid,kind,payoutMethod,giftCardEmitterAccount,group,type,description,createdAt,merchantId,isRefunded,isRefund,refundTransaction,isOrderRejected,expense,__typename,amount.currency,amount.valueInCents,amount.__typename,amountInHostCurrency.currency,amountInHostCurrency.valueInCents,amountInHostCurrency.__typename,netAmount.currency,netAmount.valueInCents,netAmount.__typename,netAmountInHostCurrency.currency,netAmountInHostCurrency.valueInCents,netAmountInHostCurrency.__typename,paymentProcessorFee.currency,paymentProcessorFee.valueInCents,paymentProcessorFee.__typename,paymentMethod.name,paymentMethod.service,paymentMethod.sourcePaymentMethod,paymentMethod.type,account.id,account.legacyId,account.name,account.slug,account.isIncognito,account.type,account.__typename,oppositeAccount.id,oppositeAccount.legacyId,oppositeAccount.name,oppositeAccount.slug,oppositeAccount.isIncognito,oppositeAccount.type,oppositeAccount.__typename,permissions.id,permissions.canRefund,permissions.canDownloadInvoice,permissions.canReject,permissions.__typename,paymentMethod,expense.id,expense.type,expense.description,expense.invoiceInfo,expense.tags,expense.payee.id,expense.payee.name,expense.payee.slug,expense.payee.type,expense.payee.__typename,expense.__typename,refundTransaction.id,payoutMethod.type,payoutMethod.name,giftCardEmitterAccount.id,giftCardEmitterAccount.name,giftCardEmitterAccount.slug,giftCardEmitterAccount.__typename,amountInHostCurrency.value
3986,b3978b93-6791-474d-91e8-2ee20a19caa7,b3978b93-6791-474d-91e8-2ee20a19caa7,EXPENSE,NaN,NaN,e344e7cb-9e21-4368-89ac-86c7af1ca89c,DEBIT,Repo and module support,2023-12-28 17:09:36.200000+00:00,914831281,False,False,NaN,False,NaN,Debit,USD,-12000,Amount,USD,-12000,Amount,USD,-12039,Amount,USD,-12039,Amount,USD,-39,Amount,NaN,NaN,NaN,NaN,dgm9bnk8-0437xqr3-kw5qvzeo-ljdayw5r,46356.0,dbatools,dbatools,False,COLLECTIVE,Collective,rmvrwng4-kj03dpb3-b8l6z57o-yl9e8xba,46358,Shawn Melton,wsmelton,False,INDIVIDUAL,Individual,8k03reyd-5agmq5rz-dyyplbwo-z7j4nxv9,False,False,False,TransactionPermissions,NaN,vjrkx5lm-nv904qjz-eanp8bwa-7zdygoe3,INVOICE,Repo and module support,,[],rmvrwng4-kj03dpb3-b8l6z57o-yl9e8xba,Shawn Melton,wsmelton,INDIVIDUAL,Individual,Expense,NaN,BANK_ACCOUNT,NaN,NaN,NaN,NaN,NaN,-120.00
3987,89ac51ae-2ff1-4786-88a3-052deedd71fe,89ac51ae-2ff1-4786-88a3-052deedd71fe,EXPENSE,NaN,NaN,101aef0d-4c40-4924-95b6-48b4ed506103,DEBIT,Developer gift,2023-12-28 17:08:55.206000+00:00,914901258,False,False,NaN,False,NaN,Debit,USD,-221386,Amount,USD,-221386,Amount,USD,-222281,Amount,USD,-222281,Amount,USD,-895,Amount,NaN,NaN,NaN,NaN,vjrkx5lm-nv904qjl-9lz68bwa-7zdygoe3,30588.0,Stellarium,stellarium,False,COLLECTIVE,Collective,53kzxy4v-07wlr6mk-rk7qmj9n-o8agdbe5,30590,Georg Zotti,georg-zotti,False,INDIVIDUAL,Individual,vrgbk357-l4x96eyv-bj4pomew-a0jdyzn8,False,False,False,TransactionPermissions,NaN,5ax8emk7-lnw54q7g-mgdqgyvj-0o93zdrb,INVOICE,Developer gift,,[],53kzxy4v-07wlr6mk-rk7qmj9n-o8agdbe5,Georg Zotti,georg-zotti,INDIVIDUAL,Individual,Expense,NaN,BANK_ACCOUNT,NaN,NaN,NaN,NaN,NaN,-2213.86
3994,3985e9cb-6e7e-4b56-806e-c8623c909db9,3985e9cb-6e7e-4b56-806e-c8623c909db9,EXPENSE,NaN,NaN,4ff8a423-6ac4-47ad-bf6b-d97875c4c01c,DEBIT,Yii 3 and yiiframework.com development,2023-12-28 16:35:15.005000+00:00,914833331,False,False,NaN,False,NaN,Debit,USD,-146400,Amount,USD,-146400,Amount,USD,-146439,Amount,USD,-146439,Amount,USD,-39,Amount,NaN,NaN,NaN,NaN,eeng0kzd-yvor4pz8-ggbpbma8-37xlw95j,30649.0,Yii Software,yiisoft,False,COLLECTIVE,Collective,rxg0j35l-zkwm6v5x-layqvoe9-8n47daby,451148,Alexey Rogachev,alexey-rogachev,False,INDIVIDUAL,Individual,eng0kzdy-vor4pzev-5lgpbma8-37xlw95j,False,False,False,TransactionPermissions,NaN,vjrkx5lm-nv904qjz-e7np8bwa-7zdygoe3,INVOICE,Yii 3 and yiiframework.com development,,[],rxg0j35l-zkwm6v5x-layqvoe9-8n47daby,Alexey Rogachev,alexey-rogachev,INDIVIDUAL,Individual,Expense,NaN,BANK_ACCOUNT,NaN,NaN,NaN,NaN,NaN,-1464.00
4008,1ea98914-c16d-48b9-ba66-80cd18df2d4c,1ea98914-c16d-48b

In [15]:
# merged_df = pd.merge(df_wise_reconciliation, df_platform_wise[['merchantId', 'id']].dropna(subset=['merchantId']).astype({'merchantId': 'int64'}), left_on='id', right_on='merchantId', how='left', suffixes=('', '_platform_transaction'))
df_platform_wise = pd.merge(df_platform_wise_related.dropna(subset=['merchantId']).astype({'merchantId': 'int'}), df_wise_reconciliation, left_on='merchantId', right_on='wise.id', how='left').dropna(subset=['wise.id'])
print('Successfully merged {} rows out of {}'.format(len(df_platform_wise), len(df_platform_wise)))

df_left_platform = df_platform_wise[~df_platform_wise['id'].isin(df_platform_wise['id'])]
df_left_wise = df_wise_reconciliation[~df_wise_reconciliation['wise.id'].isin(df_platform_wise['wise.id'])]

print(f"Platform transactions that have a corresponding Wise transaction: {len(df_platform_wise)}")
print(f"Platform transactions without a corresponding Wise transaction: {len(df_left_platform)}")
print(f"Percentage of Platform transactions that have a corresponding Wise transaction: {round(len(df_platform_wise)/len(df_platform_wise)*100, 2)}%")
print(f"Wise transactions without a corresponding platform transaction: {len(df_left_wise)}")
print(f"Percentage of Wise transactions that have a corresponding Platform transaction: {round(len(df_platform_wise)/len(df_wise_reconciliation)*100, 2)}%")


Successfully merged 3366 rows out of 3366
Platform transactions that have a corresponding Wise transaction: 3366
Platform transactions without a corresponding Wise transaction: 0
Percentage of Platform transactions that have a corresponding Wise transaction: 100.0%
Wise transactions without a corresponding platform transaction: 44
Percentage of Wise transactions that have a corresponding Platform transaction: 98.71%


In [16]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
WISE_TRACE_ACCOUNT = 'XXXXXX7206'
df_banks2 = df_banks[df_banks['Detail'].str.contains(WISE_TRACE_ACCOUNT)]
df_banks2.sample()


,index,Post Date,Customer Ref,Debit,Credit,BAI Description,Detail,Account Number
551,551,2023-02-09 00:00:00+00:00,REF 0400933L FU,250003.52,NaN,PREAUTHORIZED ACH DEBIT,REF 0400933L FUNDS TRANSFER TO DEP XXXXXX...,4869374308


In [17]:
def matchByDateAndValue(df1, df1cols, df2, df2cols, timeDelta = '10 days', valueDelta = 0.1):
    a = df1[df1cols]
    b = df2[df2cols]
    matched = []
    def naive_transaction_matcher(x):
        possible = b.drop(index=matched)
        possibleByDate = (possible[df2cols[0]] - x[df1cols[0]]).abs().sort_values(ascending=True)
        possibleByDate = possibleByDate[possibleByDate < pd.Timedelta(timeDelta)]
        if len(possibleByDate) == 0:
            return None
        possible = possible.loc[possibleByDate.index]
        possibleByAmount = (possible[df2cols[1]] - x[df1cols[1]]).abs().sort_values(ascending=True)
        possibleByAmount = possibleByAmount[possibleByAmount < possible.loc[possibleByAmount.index][df2cols[1]] * valueDelta]
        if len(possibleByAmount) == 0:
            return None
        id = int(possibleByAmount.index[0])
        matched.append(id)
        return id
    indexes = pd.DataFrame(a.apply(naive_transaction_matcher, axis=1).dropna().astype('int'), columns=["df2key"])
    return df1.join(indexes).join(df2, on="df2key")

df_wise_bank = matchByDateAndValue(df_left_wise, ['wise.created', 'wise.sourceValue'], df_banks2, ['Post Date', 'Debit'] , '2 days', 0.05).dropna(subset=['df2key'])
df_left_wise = df_left_wise[~df_left_wise['wise.id'].isin(df_wise_bank['wise.id'])]
print(f"Wise transactions with a corresponding Bank transaction: {len(df_wise_bank)}")
print(f"Wise transactions without either a corresponding Platform transaction or a Bank transaction: {len(df_left_wise)} out of {len(df_wise_reconciliation)}")
print(f"Percentage of Wise transactions that have a corresponding transaction either on the Platform or on the Bank: {round((len(df_wise_bank)+len(df_platform_wise))/len(df_wise_reconciliation)*100, 2)}%")
df_left_wise.head()

Wise transactions with a corresponding Bank transaction: 34
Wise transactions without either a corresponding Platform transaction or a Bank transaction: 10 out of 3410
Percentage of Wise transactions that have a corresponding transaction either on the Platform or on the Bank: 99.71%


,wise.id,wise.user,wise.targetAccount,wise.sourceAccount,wise.quote,wise.quoteUuid,wise.status,wise.reference,wise.rate,wise.created,wise.business,wise.transferRequest,wise.details.reference,wise.hasActiveIssues,wise.sourceCurrency,wise.sourceValue,wise.targetCurrency,wise.targetValue,wise.customerTransactionId
622975771,622975771,6421549,59807023,None,None,c0f4112b-79c9-4c5c-b2d9-d30d21c0b5bc,outgoing_payment_sent,5479Refund,1.000000,2023-03-02 12:33:30+00:00,4556280,None,5479Refund,False,USD,160000.00,USD,160000.00,bc25084e-c74e-4cfc-8c2d-ea199f9d05ac
633191295,633191295,6421549,352085054,59807023,None,3090cee4-6384-4489-bc52-9b732b02d446,outgoing_payment_sent,,0.823181,2023-03-14 15:58:20+00:00,4556280,None,,False,USD,200.00,GBP,164.64,8560b79b-2123-4ee7-8d90-bfb17b3211a4
639795561,639795561,42089850,352322636,59807023,None,126efc32-9929-4dd4-8e6f-49ce7ec855da,outgoing_payment_sent,Correction for expense number 124655,1.000000,2023-03-22 18:41:30+00:00,4556280,None,Correction for expense number 124655,False,USD,36.71,USD,36.71,126efc32-9929-4dd4-8e6f-49ce7ec855da
639801729,639801729,42089850,325692396,59807023,None,32a4afad-aac6-4dd3-9c09-d04a8f3b6b43,outgoing_payment_sent,For expense 123089,0.810898,2023-03-22 18:50:14+00:00,4556280,None,For expense 123089,False,USD,38.47,GBP,31.20,32a4afad-aac6-4dd3-9c09-d04a8f3b6b43
639805550,639805550,42089850,280958604,59807023,None,132c35ce-497f-401e-a4b4-825b27a04b53,outgoing_payment_sent,To correct payout for 125696,0.918550,2023-03-22 18:55:29+00:00,4556280,None,To correct payout for 125696,False,USD,119.25,EUR,109.54,132c35ce-497f-401e-a4b4-825b27a04b53
